In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import csv
import pandas as pd
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST driver version for 99.0.4844
Driver [C:\Users\april\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


In [2]:
def get_url(search_term):
    "Generate a url from a search term"
    
    template = "https://www.amazon.com/s?k={}"
    search_term = search_term.replace(' ','+')

    url = template.format(search_term)

    # add term query to url
    url += "&page={}&ref=sr_pg_{}"

    return url

In [3]:
def saveString(html,page):
    try:
        filename = "amazon_result_" + str(page) + ".html"
        file = open(filename,"w", encoding = 'utf-8')
        file.write(str(html))
        file.close()
    except Exception as ex:
        print('Error: ' + str(ex))

In [4]:
def loadString(f):
    try:
        html = open(f, "r", encoding='utf-8').read()
        return(html)
    except Exception as ex:
        print('Error: ' + str(ex))

In [5]:
def extract_record(item):

    # Description and URL
    atag = item.h2.a
    description = atag.text.strip()
    url = "https://www.amazon.com" + atag.get('href')

    # Current Price
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return

    # Rank and rating
    try:
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    ## All these below fields should be NONE in the entry if doesn't exist
    
    # Original Price 
    try:
        price_orig = item.find('span',{'class':'a-price a-text-price'}).text
    except AttributeError:
        price_orig = ""


    # Coupons/Discount 
    try:
        coupon = item.find('span',{'class':'s-coupon-unclipped'}).text
    except AttributeError:
        coupon = ""


    # Free Delivery/Free One Day 
    try:
        delivery = item.find('div',{'class':'a-row a-size-base a-color-secondary s-align-children-center'}).text
    except AttributeError:
        delivery = ""

    # Prime or Not
    if item.find('i',{'aria-label':"Amazon Prime"}):
        prime = 1
    else:
        prime = 0
    
    # Result Combination
    result = (description, price, price_orig, coupon, delivery, rating, review_count, prime, url)
    return result


In [6]:
# from pymongo import MongoClient
import pymongo
import bson

# connect to the mongoclient
try:
    client = pymongo.MongoClient('mongodb://localhost:27017')
    print("Connected successfully")
except:
    print("Could not connect to MongoDB")

db = client['sample']
collection = db.amazon_office_chairs

Connected successfully


In [9]:
def main(search_term):
    # startup the webdriver
    driver = webdriver.Chrome(ChromeDriverManager().install())

    records = []
    url = get_url(search_term)
    
    # Since Amazon only provides 7 pages for general search result
    for page in range(1,8):
        driver.get(url.format(page, page))
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        saveString(soup, page)

        results = soup.find_all('div', {'data-component-type':'s-search-result'})

        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)

    driver.close()

    with open('webscraping_result_test.csv', 'w', newline = '', encoding = 'utf-8' ) as f:
        writer = csv.writer(f)
        writer.writerow(['Description','Price','Price_orig','Coupon','Delivery','Rating','ReviewCount','Prime','Url'])
        writer.writerows(records)
    
    combined_df = pd.DataFrame(records)
    combined_df.columns = ['Description','Price','Price_orig','Coupon','Delivery','Rating','ReviewCount','Prime','Url']
    amz_collection = db.get_collection("amazon_office_chairs")
    amz_collection.insert_many(combined_df.to_dict('records'))

import time
time_start = time.time()
main('office chair')
time_end = time.time()
print('time_cost', time_end-time_start, 's')



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST driver version for 99.0.4844
Driver [C:\Users\april\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


time_cost 121.91902422904968 s
